In [1]:
import numpy as np
import math
from copy import copy
import sklearn.datasets
from sklearn import svm
from sklearn import metrics

In [2]:
X,y = sklearn.datasets.make_hastie_10_2()
X_train = X[0:8000,:]
y_train = y[0:8000]
X_test = X[8000:,:]
y_test = y[8000:]

# Exercise 1
Implement the AdaBoost ensemble algorithm by completing the following code:

In [3]:
import copy 

class SVC_:
    def __init__(self, kernel="rbf", degree="3"):
        self.svc = svm.SVC(kernel=kernel, degree=degree, gamma='auto')

    def fit(self, X,y,sample_weight=None):
        if sample_weight is not None:
            sample_weight = sample_weight * len(X)

        self.svc.fit(X,y,sample_weight=sample_weight)
        return self

    def predict(self, X):
        return self.svc.predict(X)

def error(predictions, labels):
    errors = 0
    for i in range(len(labels)):
        if predictions[i] != labels[i]:
            errors += 1
    return errors / len(labels)

class AdaBoost:
    def __init__(self, weakModel, T):
        self.base_model = weakModel
        self.num_iter = T
        self.models = [None]*T
        for i in range(T):
            self.models[i] = copy.deepcopy(weakModel)
        self.alpha = [0] * T
        self.K = 500
            

    def fit(self, X, y):
        y = np.array(y)        
        w = np.full(len(X), 1./float(len(X)))
        for i in range(self.num_iter):
            self.models[i].fit(X, y, sample_weight=w)
            predictions = self.models[i].predict(X)
            epsilon = 0
            for j in range(len(X)):
                if predictions[j] != y[j]:
                    epsilon += w[j]
            self.alpha[i] = 0.5 * np.log((1- epsilon) / epsilon)
            w = w * np.exp(-self.alpha[i]*y*predictions)       
            w = w / np.linalg.norm(w, 1)
            if i % self.K == 0:
                pass
                print("Training iterazione " + str(i))
                print("Errore modello corrente: " + str(error(predictions, y)))
                print("Errore ensemble: " + str(error(self.predict(X, i), y)))
        return self
            

    def predict(self, X, iterations=-1): # iterations is only for testing
        if iterations == -1:
            iterations = self.num_iter
        res = [0] * len(X)
        for i in range(iterations):
            pred = self.models[i].predict(X)
            weighted_pred = [p * self.alpha[i] for p in pred]
            for j in range(len(X)):
                res[j] += weighted_pred[j]
        res = [np.sign(r) for r in res]
        return res

In the implementation you are free to assume:
- that the problem is a binary classification problem with labels in $\{-1, +1\}$.
- that the weakModel can fit a weighted sample set by means of the call `weakModel.fit(X,y,sample_weight=w)` where `w` is a vector of length $|y|$.

Test your implementation on the dataset loaded above and using an SVC with a polynomial kernel. 

In [4]:
weakModel = SVC_(kernel="poly", degree=3)
adaboost = AdaBoost(weakModel, 100)
c = adaboost.fit(X_train, y_train)

Training iterazione 0
Errore modello corrente: 0.3435
Errore ensemble: 1.0


In [5]:
print(X_train.shape)
y_train_ = adaboost.predict(X_train)
y_test_ = adaboost.predict(X_test)

(8000, 10)


In [6]:
print("Train Error:", error(y_train, y_train_))
print("Train Accuracy:", metrics.accuracy_score(y_train, y_train_))

Train Error: 0.11625
Train Accuracy: 0.88375


In [7]:
print("Test Error:", error(y_test, y_test_))
print("Test Accuracy:", metrics.accuracy_score(y_test, y_test_))

Test Error: 0.14025
Test Accuracy: 0.85975


In [8]:
svc = svm.SVC(kernel='poly', degree=3, gamma='auto')
svc = svc.fit(X_train, y_train)
p = svc.predict(X_test)
print('Acc: ', metrics.accuracy_score(y_test, p))

Acc:  0.60225


and evaluate the AdaBoost performances as usual by calculating the classification error. 

**Note 1**:  
since the labels are bound to be in ${+1, -1}$, the classification error can be easily computed as:
$$
   error(y,y') = \frac{1}{2} - \frac{y^T \times y'}{2N},
$$
where $N$ is the total number of examples. The formula can be derived noticing that $y^T \times y'$ calculates the number $N_c$ of examples correctly classified  minus the number $N_{\bar c}$ of examples incorrectly classified. We have then $y^T \times y' = N_c - N_{\bar c}$ and by noticing that $N = N_c + N_{\bar c}$:
$$
   N - y^T \times y' = 2 N_{\bar c} \Rightarrow \frac{N - y^T \times y'}{2 N} = \frac{N_{\bar c}}{N} = error(y,y')
$$

**Note 2**:
do not forget to deepcopy your base model before fitting it to the new data

**Note 3**:
The SVC model allows specifying weights, but it *does not* work well when weights are normalized (it works well when the weights are larger). The following class takes normalized weights and denormalize them before passing them to the SVC classifier:

```python
    class SVC_:
        def __init__(self, kernel="rbf", degree="3"):
            self.svc = SVC(kernel=kernel, degree=degree)

        def fit(self, X,y,sample_weight=None):
            if sample_weight is not None:
                sample_weight = sample_weight * len(X)

            self.svc.fit(X,y,sample_weight=sample_weight)
            return self

        def predict(self, X):
            return self.svc.predict(X)
```

# Exercise 2

Write a weak learner to be used with the AdaBoost algorithm you just wrote. The weak learner that you will implement shall work as follows:

- creates a random linear model $y(x) = \mathbf{w} \cdot \mathbf{x} + t$ by generating the needed weight vector $\mathbf{w}$ and $t$ at random; each weight shall be sampled from U(-1,1);
- it evaluates the weighted loss $\epsilon_t$ on the given dataset and flip the linear model if $\epsilon_t > 0.5$
- at prediction time it predicts +1 if $\mathbf{x} \cdot \mathbf{w} > 0$ it predicts -1 otherwise.

In [9]:
class RandomLinearModel:
    def loss(self, y, y_, w = [1/len(y)] * len(y)):
        res = 0
        for i in range(len(y)):
            if y[i] != y_[i]:
                res += w[i]
        return res
        
    def fit(self,X,y,sample_weight=None):
        self.flip = False
        self.w = np.random.uniform(-1, 1, len(X[0]))
        self.t = np.random.uniform(-1, 1)
        weighted_loss = self.loss(y, self.predict(X), sample_weight)
        if weighted_loss > 0.5:
            self.flip = True
        return self
        
    def predict(self,X):
        predictions = []
        for x in X:
            s = 0
            for feature in range(len(x)):
                s += self.w[feature] * x[feature]
            predictions.append(np.sign(s + self.t))
        if self.flip:
            predictions = [-p for p in predictions]
        return predictions

Learn an AdaBoost model using the RandomLinearModel weak learner printing every $K$ iterations the weighted error and the current error of the ensemble (you are free to choose $K$ so to make your output just frequent enough to let you know what is happening but without flooding the console with messages). Evaluate the training and test error of the final ensemble model.

In [10]:
rs = RandomLinearModel()
a = AdaBoost(rs, 10000)
a.fit(X_train, y_train)

y_train_ = a.predict(X_train)
y_test_ = a.predict(X_test)

Training iterazione 0
Errore modello corrente: 0.470125
Errore ensemble: 1.0
Training iterazione 500
Errore modello corrente: 0.46875
Errore ensemble: 0.42375
Training iterazione 1000
Errore modello corrente: 0.482375
Errore ensemble: 0.390875
Training iterazione 1500
Errore modello corrente: 0.507125
Errore ensemble: 0.36575
Training iterazione 2000
Errore modello corrente: 0.527125
Errore ensemble: 0.356875
Training iterazione 2500
Errore modello corrente: 0.521
Errore ensemble: 0.34225
Training iterazione 3000
Errore modello corrente: 0.4705
Errore ensemble: 0.324125
Training iterazione 3500
Errore modello corrente: 0.49525
Errore ensemble: 0.3115
Training iterazione 4000
Errore modello corrente: 0.48525
Errore ensemble: 0.297875
Training iterazione 4500
Errore modello corrente: 0.473125
Errore ensemble: 0.288
Training iterazione 5000
Errore modello corrente: 0.53975
Errore ensemble: 0.273375
Training iterazione 5500
Errore modello corrente: 0.534125
Errore ensemble: 0.262875
Traini

In [11]:
print("Train Error:", error(y_train, y_train_))
print("Train Accuracy:", metrics.accuracy_score(y_train, y_train_))

Train Error: 0.194125
Train Accuracy: 0.805875


In [12]:
print("Test Error:", error(y_test, y_test_))
print("Test Accuracy:", metrics.accuracy_score(y_test, y_test_))

Test Error: 0.438
Test Accuracy: 0.562


Write few paragraphs about what you think about the experiment and about the results you obtained. 

As expected AdaBoost greatly increased the performance of the linear algorithms. When using the SVM with cubic kernel we get comparable accuracy on both training and test set, always much higher than the accuracy of the weak learner alone.
Using the random linear model as a weak learner we can get an acceptable error on the training set through 10000 iterations. This shows the AdaBoost capability as a bias reduction machine. But the error on the test set is almost as high as the random model: that is because the error reduction applies only to the training examples and, while it can carry over to new examples, this experiment shows it's not always the case.